In [ ]:
import warnings
warnings.filterwarnings("ignore") 
import arviz as az
import numpy as np
import pandas as pd
import pickle

# Set Plotly as default plotting backend for pandas
pd.options.plotting.backend = "plotly"  


from tb_incubator.input import load_param_info
from tb_incubator.constants import set_project_base_path
from tb_incubator.calibrate import (
    calculate_xpert_scenario_outputs,
    calculate_detection_scenario_outputs,
    calculate_combined_detection_xpert_outputs,
    calculate_acf_scenario_outputs,
    calculate_combined_xpert_acf_scenario_outputs,
)

# Set project base path
project_paths = set_project_base_path("../tb_incubator/")


In [ ]:
calib_out = project_paths["OUT_PATH"]
output_path = project_paths["OUTPUTS"]
file_suffix = "xpert_utilisation_improvement_p17_96"

In [ ]:
# extract data - only run for first time
idata = az.from_netcdf(calib_out / f'calib_full_out_{file_suffix}.nc')

In [ ]:
burnt_idata = idata.sel(draw=np.s_[2500:])
idata_extract = az.extract(burnt_idata, num_samples=1000)
inference_data = az.convert_to_inference_data(idata_extract.reset_index('sample'))
az.to_netcdf(inference_data, calib_out / f'inference_data_{file_suffix}.nc')

In [ ]:
# Load saved idata
idata = az.from_netcdf(calib_out / f'inference_data_{file_suffix}.nc')

In [ ]:
param_info = load_param_info()
fixed_params = param_info["value"]
covid_effects = {
    'detection_reduction':True
}

In [ ]:
basecase_xpert = calculate_xpert_scenario_outputs(fixed_params, idata, covid_effects=covid_effects)
with open(output_path / f'xpert_quant_outputs.pkl', 'wb') as f:
     pickle.dump(basecase_xpert, f)

In [ ]:
basecase_detection = calculate_detection_scenario_outputs(fixed_params, idata, covid_effects=covid_effects)
with open(output_path / f'detection_quant_outputs.pkl', 'wb') as f:
     pickle.dump(basecase_detection, f)

In [ ]:
basecase_combined = calculate_combined_detection_xpert_outputs(fixed_params, idata, covid_effects=covid_effects)
with open(output_path / f'combined_quant_outputs.pkl', 'wb') as f:
     pickle.dump(basecase_combined, f)

In [ ]:
percentages = [5, 10, 20, 30, 40, 50, 60, 70, 80]  
acf_configs = {
    f"acf_{pct}": {
        2026.0: 0.0,
        2026.01: round(float(-np.log(1 - pct/100)), 2),
        2027.0: round(float(-np.log(1 - pct/100)), 2),
        2027.01: 0.0
    }
    for pct in percentages
}

In [ ]:
basecase_acf = calculate_acf_scenario_outputs(fixed_params, idata, acf_rate_configs=acf_configs, frequency=None, covid_effects=covid_effects)

In [ ]:
with open(output_path / f'acf_quant_outputs.pkl', 'wb') as f:
     pickle.dump(basecase_acf, f)

In [ ]:
acf_outputs = {}
for coverage in [0.2, 0.4, 0.8]:
    rate = -np.log(1 - coverage)
    acf_outputs[coverage] = calculate_acf_scenario_outputs(
        fixed_params, idata, covid_effects=covid_effects, acf_rate=rate
    )


In [ ]:
with open(output_path / f'acf_freq_quant_outputs.pkl', 'wb') as f:
     pickle.dump(acf_outputs, f)

In [ ]:
basecase_combined_acf_xpert = calculate_combined_xpert_acf_scenario_outputs(
    fixed_params,
    idata,
    covid_effects=covid_effects,
    acf_rate_configs=acf_configs
)

In [ ]:
with open(output_path / f'comb_acf_xpert_quant_outputs.pkl', 'wb') as f:
     pickle.dump(basecase_combined_acf_xpert, f)